# Create your own Hong Kong Stock Data Set

In this module, we are going to apply some of the things we have learned about APIs, string formatting, loops, file I/O, and Pandas. 
You will have the ability to fetch your own stock data, preprocess them, and store each company's values in a csv file.

## Getting our companies of interest

Let's assume we want to fetch the Data of the Hong Kong GEM board. Now what we could do is brute force ourselves through all possible numbers and see if this stock code exists. That would be time consuming, so we should know where to go for valid information:

https://www.hkex.com.hk/eng/services/trading/securities/securitieslists/ListOfSecurities.xlsx

We can extract our GEM companies by filtering, and would want to save our file so we can use the codes for our API calls as "gem_comps.xlsx".

In [1]:
import requests #for our GETting of the stock information
import pandas as pd #we want to handle the output of the API right away so there is going to be a clean dataset
import time #we need our API calls to be a little apart from each other in case we make a bulk call
import os #for file and directory handling

Note that for this notebook, we have saved the file in the folder 'stock_dfs'. If it does not exist, we can create it upon running our script, and download the file right to this location:

In [7]:
if not os.path.exists('stock_dfs/GEM'):
    os.makedirs('stock_dfs/GEM')

Now we can load our table with the necessary information an create a list of valid ticker numbers for our API:

In [2]:
gemtable = pd.read_excel('stock_dfs/gem_comps.xlsx')
tickers = gemtable["CODE"].tolist()

In [3]:
len(tickers)

379

In [13]:
tickers[0]

8001

## Now let's see what a single equity output is like:

In [4]:
ticker = tickers[0]

In [5]:
link = ('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}.HK&outputsize=compact&apikey=XHUW4LB158S6333F'.format(ticker))
re = requests.get(link)

Upon calling re.json() we can see that our result is a json file, meaning we can handle it like a dictionary in Python. Each row represents a trading day, and the necessary information that we want in our columns is shown as values for each day. There is also some Meta Data which we wouldn't want to use in our case, especially since we're about to do some quantified analyses with our results. Let's turn our response into a data frame and get rid of the unnecessary information:

In [8]:
df_recent = pd.DataFrame.from_dict(re.json()) #parse the json information to a dataframe
df_recent.drop(['Meta Data'], axis =1, inplace = True) #drop the meta data
df_recent.drop(df_recent.index[[0,1,-1,-2,-3]], inplace = True) #drop the trailing and leading rows containing non quantified information
df_recent.reset_index(inplace = True) #make sure the dates are not our index

In [9]:
df_recent.head()

,index,Time Series (Daily)
0,2018-04-18,"{'1. open': '0.7700', '2. high': '0.8000', '3...."
1,2018-04-19,"{'1. open': '0.7800', '2. high': '0.7800', '3...."
2,2018-04-20,"{'1. open': '0.7700', '2. high': '0.7900', '3...."
3,2018-04-23,"{'1. open': '0.7700', '2. high': '0.8200', '3...."
4,2018-04-24,"{'1. open': '0.8300', '2. high': '0.8500', '3...."


The next step would be to unpack the information that is nested in the TimeSeries (Daily) column's dictionaries:

In [10]:
ohlc = pd.DataFrame(df_recent['Time Series (Daily)'].dropna().tolist()) # turn the column's information into a series, drop NaN that have been carried along
df_recent = pd.merge(df_recent, ohlc, left_index=True, right_index=True) # glue that series onto the existing dataframe
df_recent.drop(['Time Series (Daily)'], axis = 1, inplace = True) # drop the column with the wrapped information

In [11]:
df_recent.head()

,index,1. open,2. high,3. low,4. close,5. volume
0,2018-04-18,0.7700,0.8000,0.7700,0.7800,1352000
1,2018-04-19,0.7800,0.7800,0.7700,0.7800,1304000
2,2018-04-20,0.7700,0.7900,0.7700,0.7800,1336000
3,2018-04-23,0.7700,0.8200,0.7700,0.8100,1756000
4,2018-04-24,0.8300,0.8500,0.8000,0.8000,1356000


Let's save the file with a clear filename as a csv:

In [53]:
df_recent.to_csv('stock_dfs/GEM/{}.csv'.format(ticker))

In [13]:
test = pd.read_csv('stock_dfs/GEM/{}.csv'.format(ticker))

In [14]:
test.plot(y='4. close', legend = False)

## Assignment
Write a function that fetches the data for all GEM companies. Try to implement an exception handler for cases that cannot be fetched (delistings, for example).
Your function should be able to check whether your directory already contains information about the share, and only append new trading days.